# Comparison between original model and model with variable service time

In order to keep the same average capacity on both models (instant service and variable service time), the maxium number of available servers must be equal to 
$S_d$

$$S_d = t_m \cdot S_o$$

where $t_m$ is the average service time and $S_o$ is the maximum number of servers in the model with instant service.

For a maxium service time of 4 steps ($t_{serM} = 3-1$), with service times as a uniform distribuition,$t_m = 3$. For a valid comutation of $S_d$, $t_m$ must be and integer. Therefore, $t_{serM}$ must be odd.

$$t_m = \frac{((t_{serM}+1)(t_{serM}+2)/2)-1}{t_{serM}} = \frac{2+3+4}{3} = 3 $$

In [ ]:
using JuMP
using HiGHS
using Ipopt
using Juniper
using DelimitedFiles
using Serialization
using Random

include("utils.jl") 
include("optimize_list.jl")

### Exogenous variables

In [ ]:
N = 10           # number of examples
horiz = 20       # total horizon

limit_count = 10 # max number of trials to create a new replacement demand in case of infeasibility

createDemands = false

d_prop = distribution_properties(15, 0, "uniform"); # properties of the demand distribution
a_prop = distribution_properties(1, 0, "uniform");

if createDemands
    d_mat = demand_generator_mat(N, horiz, d_prop.M, d_prop.type, d_prop.std_dev);
    a_mat = demand_generator_mat(N, horiz, a_prop.M, a_prop.type, a_prop.std_dev);
else
    d_fn = string("exogenous_vars//d_mat_", d_prop.type, "_max_", d_prop.M, "_dev_", d_prop.std_dev, ".txt");
    a_fn = string("exogenous_vars//a_mat_", a_prop.type, "_max_", a_prop.M, "_dev_", a_prop.std_dev, ".txt");
    d_mat = DelimitedFiles.readdlm(d_fn);
    a_mat = DelimitedFiles.readdlm(a_fn);
end;

### Parametrers

In [ ]:
compensateSerDelay = true
compensateCostDelay = true

# bounds
XM = 6            # max queue length 
YM = 10           # max buffer length before dropping calls
phiM = 9          # max adimission to queue
serM_nd = 5       # max number of servers original model
tserM = 3         # max delay in service time

# iniital conditions
X0 = 4
Y0 = 6
L0 = 0
Z0 = 1
tm = (((tserM+1)*(tserM+2)/2)-1)/tserM # average service time

# Non-linear objective function costs
c_blr_nd = 400
c_ser_nd = 1

# Linear objective function costs
if compensateCostDelay
    c_Z_d = 3/2/tm      # weight of served clients      
    c_ser_d = 1/tm      # weight of server cost
    c_L_d = 1/2/tm      # weight of buffer cost
else
    c_Z_d = 3/2               
    c_ser_d = 1         
    c_L_d = 1/2         
end

# Adjust max number of servers for the model with delay
if compensateSerDelay
    serM_d = Int(serM_nd*tm)                  
else
    serM_d = serM_nd                        
end

bds_nd = bounds(XM, YM, phiM, serM_nd, 0)
bds_d = bounds(XM, YM, phiM, serM_d, tserM)

    
ic = initial_conditions(X0, Y0, L0, Z0)

c_nd = c(0, 0, c_ser_nd, c_blr_nd)  # original model costs
c_d = c(c_Z_d, c_L_d, c_ser_d, c_blr_nd)  # model with delay costs

df_input = df_input_generator(horiz, bds_d.serM, bds_d.tserM);

### Model with instant service

In [ ]:
# Linearized objetive function
result_os_nd_lin, d_mat_os_nd_lin, a_mat_os_nd_lin, count_os_nd_lin = optimize_list("nd_os_lin", d_mat, a_mat);
result_om_nd_lin, d_mat_om_nd_lin, a_mat_om_nd_lin, count_om_nd_lin = optimize_list("nd_om_lin", d_mat, a_mat);

In [ ]:
# Non linnear objective function
result_os_nd, d_mat_os_nd, a_mat_os_nd, count_os_nd = optimize_list("nd_os", d_mat, a_mat);
result_om_nd, d_mat_om_nd, a_mat_om_nd, count_om_nd = optimize_list("nd_om", d_mat, a_mat);

### Model with delay

In [ ]:
# Linearized objetive function
result_os_d, d_mat_os_d, a_mat_os_d, count_os_d = optimize_list("d_os", d_mat, a_mat);
result_om_d, d_mat_om_d, a_mat_om_d, count_om_d = optimize_list("d_om", d_mat, a_mat);

In [ ]:
# Non linnear objective function
result_os_d_nonlin, d_mat_os_d_nonlin, a_mat_os_d_nonlin, count_os_d_nonlin = optimize_list("d_os_nonlin", d_mat, a_mat);
result_om_d_nonlin, d_mat_om_d_nonlin, a_mat_om_d_nonlin, count_om_d_nonlin = optimize_list("d_om_nonlin", d_mat, a_mat);

## Results comparison

In [ ]:
# results_list = [result_os_nd_lin, result_om_nd_lin, result_os_nd, result_om_nd, result_os_d, result_om_d, result_os_d_nonlin]
# a_mat_list = [a_mat_os_nd_lin, a_mat_om_nd_lin, a_mat_os_nd, a_mat_om_nd, a_mat_os_d, a_mat_om_d, a_mat_os_d_nonlin]
# d_mat_list = [d_mat_os_nd_lin, d_mat_om_nd_lin, d_mat_os_nd, d_mat_om_nd, d_mat_os_d, d_mat_om_d, d_mat_os_d_nonlin]
# count_list = [count_os_nd_lin, count_om_nd_lin, count_os_nd, count_om_nd, count_os_d, count_om_d, count_os_d_nonlin]

results_list = [result_os_d]
a_mat_list = [a_mat_os_d]
d_mat_list = [d_mat_os_d]
count_list = [count_os_d]

res_stats = Dict()

for i in 1:length(results_list)
    res = results_list[i]
    result_id = res.id*"_"*string(horiz)*"_"*string(d_prop.M)*"_"*string(a_prop.M)*"_"*string(res.bds.XM)*"_"*string(res.bds.YM)*"_"*string(res.bds.phiM)*"_"*string(res.bds.serM)*"_"*string(res.bds.tserM)
    cost, av_blr, av_dr, av_ser, total_clients = compute_metrics(res, horiz, N)

    # Print results table
    println(result_id)
    data = hcat(av_blr, total_clients, av_dr, av_ser, cost)
    header = (["average blr", "Average served clients", "Average dropped clients", "Average active Servers", "Average cost function value"]);
    printTable(data, header)

    # add csv line
    # add_line_to_csv(result_id, res, horiz, d_prop, a_prop, cost, av_blr, av_dr, av_ser, total_clients, count_list[i])

    # Save results to file
    var_dict = Dict("res"=>res, "a"=>a_mat_list[i], "d"=>d_mat_list[i])
    fn = string("results//",result_id,".txt")
    serialize(fn, var_dict)
end

### Create demands

In [ ]:
createDemands = true

N = 15            # number of examples
horiz = 50        # total horizon
 
M_list = [15]  # max demand
# M_list = [1,3]

std_dev_list = [0]

type_distr = "uniform"
exo_var = "d"

for M in M_list
    if type_distr == "uniform"
        std_dev_list = [0]
    end
    for std in std_dev_list
        d_mat = demand_generator_mat(N, horiz, M, type_distr, std)
        d_mat_fn = "exogenous_vars//"* exo_var*"_mat_"* type_distr* "_max_"* string(M)* "_dev_"* string(std)* ".txt"
        writedlm(d_mat_fn, d_mat)
    end
end
